# Getting Started

In [1]:
# imports and setup
import os

import tasty.templates as tt
import tasty.constants as tc
import tasty.graphs as tg

## Initializations

We will start by loading in an ontology.  This returns an RDF graph, with typical namespaces loaded depending on which ontology (Brick or Haystack) we decided to load, but we don't need to worry about that too much.

In [ ]:
haystack_ont = tg.load_ontology('Haystack', '3.9.9')
brick_ont = tg.load_ontology('Brick', '1.1')
print(type(haystack_ont))
print(type(brick_ont))

## Namespacing Terms
We will explore some core capabilities of templates by getting some typing information from a Haystack point type.  The first thing to do is to find the correct namespaces for the terms.

_Note: The way we use Namespaced terms is a Tuple data structure as: (Namespace, term)_

__Function__: `tt.get_namespaced_terms`

__Purpose__: resolve terms to namespaces in the ontology

__Description__: The function can take a simple string, with terms delimited by '-' (likely for Haystack). A Brick point type likely will just be expressed as a single class. It can also take a dictionary.  This is used when we are trying to express Datatype Properties (fields). A datatype property is a property of an entity that 'has a value'.  At this point in time, this is only really possible in Haystack (Brick I think is still in process). This has taken inspiration from the Hayson specification. When expressing fields:
- The key must be a term in the ontology
- `_kind` is a special key.  the value will be resolved to a namespaced term (i.e. number -> ph:number)
- `val` is a special key.  it expresses the desired value of the property. A NoneType (`None`) value is used to express that this field is desired, but we don't care what the value is.

In [ ]:
# Recommend not putting 'fields' in the point type string (curVal), but we'll get back to this later.
point_type_string = 'cur-writable-motor-run-curVal-sensor-point'
haystack_namespaced_terms = tt.get_namespaced_terms(haystack_ont, point_type_string)

brick_type = 'Damper_Position_Command'
brick_namespaced_terms = tt.get_namespaced_terms(brick_ont, brick_type)

# expressing datatype properties via a dict
fields = {
    'curVal': {
        '_kind': 'number',
        'val': None
    },
    # another form
    'unit': 'cfm'
}

namespaced_fields = tt.get_namespaced_terms(haystack_ont, fields)

# types have two values in each tuple: (Namespace, term)
print(brick_namespaced_terms)

# fields have three values in each tuple: (Namespace, term, dict)
print(namespaced_fields)

## Determining Classes

The point of using Semantic Web technologies is to encode the 'type' or 'class' of something into the ontology. From the perspective of Brick, we have likely already done this by getting a namespaced term.  With Haystack, typing is a bit more nuanced, and refers more to the expected functionality (i.e. describing what it _implements_ as well as what it is).  For example, folks used to using sets of markers for expressing the point type above (`cur-air-writable-motor-curVal-sensor-point`) would know they expect to be able to:
- get the current value of this point - it is a `phIoT:cur-point`
- write to this point - it is a `phIoT:writable-point`
- it somehow also represents a type of thing that is a motor - `phIoT:motor`

We solve this with a simple function call:

__Function__: `tt.hget_entity_classes`

__Purpose__: determine (from a set of namespaced terms) the different classes implemented.

In [ ]:
structured_terms = tt.hget_entity_classes(haystack_ont, haystack_namespaced_terms)

A dictionary is returned with three keys, describing the information determined:
- 'classes'
- 'markers'
- 'fields'

Print them out to see the exact information contained in each

In [ ]:
print(structured_terms['classes'])

In [ ]:
print(structured_terms['markers'])

In [ ]:
# Here is where we pull out namespaced terms that are not things, but which are expected to contain values.
# They are automatically assigned a 'val': None
print(structured_terms['fields'])